<a href="https://colab.research.google.com/github/longhoag/PSS-Project/blob/main/PSS_Project_CS3560.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##PSS
##Team 16

In [1]:
# import dependencies
from datetime import datetime, timedelta
from typing import List, Optional
import json

## Model

In [2]:
class Task:
  # initialize attributes
  def __init__(self, task_name: str, task_type: str, start_time: float, duration: float):
    self.task_name = task_name
    self.task_type = task_type
    self.start_time = start_time
    self.duration = duration

  def get_end_time(self) -> float:
    return self.start_time + self.duration

  def get_date(self) -> Optional[int]: #displaying date for view tasks
    """Default: No specific date for a generic task."""
    return None

  def __repr__(self):
    return f"{self.task_name} ({self.task_type}): {self.start_time} - {self.get_end_time()} hrs"

# Classes Extending Task Class
class RecurringTask(Task):
  # initialize attributes
  def __init__(self, task_name: str, task_type: str, start_time: float, duration: float, start_date: int, end_date: int, frequency: int):
    super().__init__(task_name, task_type, start_time, duration)
    self.start_date = start_date
    self.end_date = end_date
    self.frequency = frequency
    self.removed_dates = set()  # to track removed occurrences

  # this methods is used to display schedule by period
  def get_occurrences_in_range(self, start_date: int, end_date: int) -> List[str]:
    """Returns occurrences within a specific date range."""
    current_date = datetime.strptime(str(self.start_date), "%Y%m%d")
    end_date_obj = datetime.strptime(str(self.end_date), "%Y%m%d")
    occurrences = []
    range_start = datetime.strptime(str(start_date), "%Y%m%d")
    range_end = datetime.strptime(str(end_date), "%Y%m%d")
    # extract valid time
    while current_date <= end_date_obj:
      if range_start <= current_date <= range_end:
        date_str = current_date.strftime("%B %d, %Y") #format
        if date_str not in self.removed_dates:
          occurrences.append(date_str)
      current_date += timedelta(days=self.frequency)

    return occurrences
  # get all the occurences
  def get_all_occurrence_dates(self) -> List[str]:
    """Returns all dates when the recurring task occurs, excluding removed dates."""
    current_date = datetime.strptime(str(self.start_date), "%Y%m%d")
    end_date_obj = datetime.strptime(str(self.end_date), "%Y%m%d")
    dates = []
    while current_date <= end_date_obj:
      date_str = current_date.strftime("%B %d, %Y")
      if date_str not in self.removed_dates:
        dates.append(date_str)
      current_date += timedelta(days=self.frequency)
    return dates

  # remove ref when add anti task
  def remove_occurrence(self, date: int) -> bool:
    """Removes a specific occurrence by date."""
    date_str = datetime.strptime(str(date), "%Y%m%d").strftime("%B %d, %Y")
    if date_str in self.get_all_occurrence_dates():
      self.removed_dates.add(date_str)
      return True
    return
  # Checks if a specific date has been removed by an AntiTask.
  def is_date_removed(self, date: int) -> bool:
    date_str = datetime.strptime(str(date), "%Y%m%d").strftime("%B %d, %Y")
    return date_str in self.removed_dates

  def get_date(self) -> Optional[int]:
    return self.start_date

class TransientTask(Task):
  def __init__(self, task_name: str, task_type: str, start_time: float, duration: float, date: int):
    super().__init__(task_name, task_type, start_time, duration) # attributes
    self.date = date

  def get_date(self) -> Optional[int]:
    return self.date

class AntiTask(Task):
  def __init__(self, task_name: str, task_type: str, start_time: float, duration: float, date: int, target_recurring_task: RecurringTask):
    super().__init__(task_name, task_type, start_time, duration) # attributes
    self.date = date
    self.target_recurring_task = target_recurring_task

  def get_date(self) -> Optional[int]:
    return self.date


In [3]:
class Schedule:
  def __init__(self):
    self.tasks: List[Task] = []  # Main list for recurring and transient tasks

  def add_task(self, task: Task) -> bool:
    if isinstance(task, AntiTask):
      return self._handle_anti_task(task)

    # Check for overlaps while allowing removed occurrences to be used
    if not TaskValidator.check_overlap(task, self):
      return False

    # prevent overlapping tasks unless it's an AntiTask
    '''
    if any(t for t in self.tasks if not isinstance(t, AntiTask) and
        t.start_time < task.get_end_time() and task.start_time < t.get_end_time()):
      return False
      '''

    self.tasks.append(task)
    return True

  def _handle_anti_task(self, anti_task: AntiTask) -> bool:
    """Cancels one specific occurrence of a RecurringTask."""
    target_task = anti_task.target_recurring_task
    if not isinstance(target_task, RecurringTask):
      return False

    # remove the specific occurrence from the RecurringTask
    if target_task.remove_occurrence(anti_task.date):
      #self.anti_tasks.append(anti_task)  # Log the anti-task for reference
      return True
    return False
    '''
    # find occurrences matching the AntiTask date
    occurrences = target_task.get_all_occurrence_dates()
    anti_task_date_str = datetime.strptime(str(anti_task.date), "%Y%m%d").strftime("%B %d, %Y")

    if anti_task_date_str in occurrences:
      self.tasks.append(anti_task)
      return True
    return False
    '''

  def delete_task(self, task_name: str) -> bool:
    for task in self.tasks:
      if task.task_name == task_name:
        self.tasks.remove(task) # delete task from list
        return True
    return False

  def edit_task(self, old_task_name: str, new_task: Task) -> bool:
    old_task = self.find_task(old_task_name) # edit by find, remove the old one, and add new one
    if old_task and self.add_task(new_task):
      self.delete_task(old_task_name)
      return True
    return False

  def find_task(self, task_name: str) -> Optional[Task]:
    for task in self.tasks:
      if task.task_name == task_name: # find through task name
        return task
    return None

  def view_tasks_by_period(self, start_date: int, period: str) -> List[Task]:
    """Retrieves tasks for a specific period."""
    relevant_tasks = []
    range_end = start_date

    # Determine range_end based on the period
    if period == "day":
      range_end = start_date
    elif period == "week":
      range_end = (datetime.strptime(str(start_date), "%Y%m%d") + timedelta(days=6)).strftime("%Y%m%d")
    elif period == "month":
      range_end = (datetime.strptime(str(start_date), "%Y%m%d") + timedelta(days=30)).strftime("%Y%m%d")
    else:
      print("Invalid period. Please choose 'day', 'week', or 'month'.")
      return relevant_tasks

    range_end = int(range_end)

    # display all the occurences
    for task in self.tasks:
      if isinstance(task, RecurringTask):
        occurrences = task.get_occurrences_in_range(start_date, range_end)
        if occurrences:
          relevant_tasks.append({
            "task": task,
            "occurrences": occurrences
          })
      elif hasattr(task, 'date') and start_date <= task.date <= range_end:
        relevant_tasks.append(task)

    return relevant_tasks

  def write_schedule_to_file(self, filename: str) -> bool: # write schedule in the format of json file
    try:
      with open(filename, 'w') as file:
        serialized_tasks = []
        for task in self.tasks:
          task_dict = task.__dict__.copy()

          # Add task_type explicitly
          task_dict["task_type"] = type(task).__name__

          # Serialize removed_dates for RecurringTask
          if isinstance(task, RecurringTask):
            task_dict["removed_dates"] = list(task.removed_dates)

          serialized_tasks.append(task_dict)

        json.dump(serialized_tasks, file, indent=4)
      return True
    except Exception as e:
      print(f"Error writing to file: {e}")
      return False

  def read_schedule_from_file(self, filename: str) -> bool:
    try:
      with open(filename, 'r') as file:
        tasks = json.load(file) # load json file

      self.tasks = []
      for task_dict in tasks:
        task_type = task_dict.pop('task_type', None)

        # Reconstruct RecurringTask with removed_dates
        if task_type == "RecurringTask":
          removed_dates = set(task_dict.pop("removed_dates", []))
          task = RecurringTask(task_type=task_type, **task_dict)
          task.removed_dates = removed_dates
        elif task_type == "TransientTask":
          task = TransientTask(task_type=task_type, **task_dict)
        elif task_type == "AntiTask":
          task = AntiTask(task_type=task_type, **task_dict)
        else:
          task = Task(task_type=task_type, **task_dict)

        self.tasks.append(task)

      return True
    except Exception as e:
      print(f"Error reading from file: {e}")
      return False

  def _reconstruct_task(self, task_dict: dict) -> Task:
    task_type = task_dict['task_type']
    if task_type == "RecurringTask":
      return RecurringTask(**task_dict)
    elif task_type == "TransientTask":
      return TransientTask(**task_dict)
    elif task_type == "AntiTask":
      return AntiTask(**task_dict)
    return Task(**task_dict)

## Task Validator

In [4]:
class TaskValidator:
  @staticmethod
  def validate_task_name(name: str, schedule: Schedule) -> bool:
    return not any(task.task_name == name for task in schedule.tasks)

  @staticmethod
  def check_overlap(task: Task, schedule: Schedule) -> bool:
    task_date = task.get_date()  # The date of the task being checked
    for t in schedule.tasks:
      # Skip RecurringTask dates removed by AntiTask
      if isinstance(t, RecurringTask) and t.is_date_removed(task.get_date()):
        continue

      # For RecurringTask, check each occurrence date
      if isinstance(t, RecurringTask):
        if task_date in [datetime.strptime(date_str, "%B %d, %Y").strftime("%Y%m%d")
                        for date_str in t.get_all_occurrence_dates()]:
          # Check time overlap if the dates match
          if t.start_time < task.get_end_time() and task.start_time < t.get_end_time():
            return False

      # General overlap check
      # For TransientTask or AntiTask, check date and time
      elif hasattr(t, 'date') and t.date == task_date:
        if t.start_time < task.get_end_time() and task.start_time < t.get_end_time():
          return False

    return True

  @staticmethod
  def validate_date(date: int) -> bool: # validate proper date format
    try:
      datetime.strptime(str(date), "%Y%m%d")
      return True
    except ValueError:
      return False

  @staticmethod
  def validate_recurring_task(task: RecurringTask) -> bool:
    return task.start_date < task.end_date

## Views

In [5]:
class TaskView:
  @staticmethod
  def display_task_details(task: Task) -> None:
    date = task.get_date()
    # display format for recurring task
    if isinstance(task, RecurringTask):
      date_str = datetime.strptime(str(task.start_date), "%Y%m%d").strftime("%B %d, %Y")
      end_date_str = datetime.strptime(str(task.end_date), "%Y%m%d").strftime("%B %d, %Y")
      all_dates = task.get_all_occurrence_dates()
      print(f"Task: {task.task_name} ({task.task_type})")
      print(f"Start Date: {date_str}")
      print(f"End Date: {end_date_str}")
      print(f"Occurrences: {', '.join(all_dates)}")
      print(f"Start Time: {task.start_time} o'clock")
      print(f"End Time: {task.get_end_time()} o'clock")
      print(f"Duration: {task.duration} hrs")
    # display format for other classes
    elif date:
      date_str = datetime.strptime(str(date), "%Y%m%d").strftime("%B %d, %Y")
      print(f"Task: {task.task_name} ({task.task_type})")
      print(f"Date: {date_str}")
      print(f"Start Time: {task.start_time} o'clock")
      print(f"End Time: {task.get_end_time()} o'clock")
      print(f"Duration: {task.duration} hrs")
    else:
      print(f"Task: {task}")

  @staticmethod
  def display_task_list(tasks: List[Task]) -> None:
    for task in tasks:
      print(task) # display whole list

  @staticmethod
  def display_error_message(message: str) -> None:
    print(f"Error: {message}")

  @staticmethod
  def display_success_message(message: str) -> None:
    print(f"Success: {message}")

In [6]:
class ScheduleView:
  @staticmethod
  def display_schedule(tasks: List[Task], period: str) -> None:
    print(f"\nSchedule for {period}:")
    if not tasks:
      print("No tasks found.")
      return

    for task in tasks:
      date = task.get_date()
      # display format for recurring task
      if isinstance(task, RecurringTask):
        date_str = datetime.strptime(str(task.start_date), "%Y%m%d").strftime("%B %d, %Y")
        end_date_str = datetime.strptime(str(task.end_date), "%Y%m%d").strftime("%B %d, %Y")
        all_dates = task.get_all_occurrence_dates()
        print(f"{task.task_name} ({task.task_type})")
        print(f"  Start Date: {date_str}")
        print(f"  End Date: {end_date_str}")
        print(f"  Occurrences: {', '.join(all_dates)}")
        print(f"  Start Time: {task.start_time} o'clock")
        print(f"  End Time: {task.get_end_time()} o'clock")
        print(f"  Duration: {task.duration} hrs")
      # display format for other task
      elif date:
        date_str = datetime.strptime(str(date), "%Y%m%d").strftime("%B %d, %Y")
        print(f"{task.task_name} ({task.task_type}) - {date_str}")
        print(f"  Start Time: {task.start_time} o'clock")
        print(f"  End Time: {task.get_end_time()} o'clock")
        print(f"  Duration: {task.duration} hrs")
      else:
        print(task)

## Controller

In [7]:
# the methods follow the class diagram
class TaskController:
  def __init__(self, schedule: Schedule, task_view: TaskView, schedule_view: ScheduleView):
    self.schedule = schedule
    self.task_view = task_view
    self.schedule_view = schedule_view

  def add_task(self):
    print("\n-- Add Task --")
    print("1. Recurring Task")
    print("2. Transient Task")
    print("3. Anti-Task")
    task_type_option = input("Choose task type: ") # option to choose from 3 types

    task_type_map = {"1": "RecurringTask", "2": "TransientTask", "3": "AntiTask"}
    task_type = task_type_map.get(task_type_option)

    if not task_type:
      print("Invalid task type selected.")
      return

    task_details = self.input_task_details(task_type)
    if task_details:
      self.create_task(task_type, task_details)

  def create_task(self, task_type: str, attributes: dict) -> None:
    task_classes = {
      "RecurringTask": RecurringTask,
      "TransientTask": TransientTask,
      "AntiTask": AntiTask,
    }
    task_class = task_classes.get(task_type)
    # invalid type
    if not task_class:
      self.task_view.display_error_message("Invalid task type.")
      return

    task = task_class(**attributes)

    # error relates to anti task
    if isinstance(task, AntiTask):
      if not isinstance(task.target_recurring_task, RecurringTask):
        self.task_view.display_error_message("Error: Target task is not a valid RecurringTask.")
        return

    # error overlap
    if TaskValidator.validate_task_name(task.task_name, self.schedule):
      if self.schedule.add_task(task):
        self.task_view.display_success_message("Task created successfully.")
      else:
        if isinstance(task, AntiTask):
          self.task_view.display_error_message("Error: Failed to cancel the recurring task.")
        else:
          self.task_view.display_error_message("Error: Task overlaps with an existing task.")
    else:
      self.task_view.display_error_message("Error: Task name already exists.")


  def edit_task(self):
    print("\n-- Edit Task --")
    task_name = input("Enter the name of the task to edit: ")
    existing_task = self.schedule.find_task(task_name)

    if not existing_task:
      print("Task not found.")
      return

    print(f"Editing task: {existing_task}")
    task_type = type(existing_task).__name__

    updated_details = self.input_task_details(task_type)
    if updated_details:
      updated_task = globals()[task_type](**updated_details)
      if self.schedule.edit_task(task_name, updated_task):
        self.task_view.display_success_message("Task edited successfully.")
      else:
        self.task_view.display_error_message("Failed to edit task. Possible overlap or validation issue.")

  # use method from model and validator class
  def delete_task(self, name: str):
    """Deletes a task by name if it exists, otherwise shows an error."""
    task = self.schedule.find_task(name)
    if task:
      if self.schedule.delete_task(name):
        self.task_view.display_success_message(f"Task '{name}' deleted successfully.")
      else:
        self.task_view.display_error_message(f"Failed to delete task '{name}'.")
    else:
      self.task_view.display_error_message(f"Task '{name}' not found.")

  # use method from model and validator class
  def find_task(self):
    print("\n-- Find Task --")
    task_name = input("Enter the name of the task to find: ")
    task = self.schedule.find_task(task_name)
    if task:
      self.task_view.display_task_details(task)
    else:
      print("Task not found.")

  def view_tasks(self):
    print("\n-- All Tasks --")
    self.schedule_view.display_schedule(self.schedule.tasks, "all")

  # use method from model and validator class
  def input_task_details(self, task_type: str): # each type of task requires their attributes
    task_name = input("Enter task name: ")
    start_time = float(input("Enter start time (e.g., 9.5 for 9:30 AM): "))
    duration = float(input("Enter duration (in hours): "))

    if task_type == "RecurringTask":
      start_date = int(input("Enter start date (YYYYMMDD): "))
      end_date = int(input("Enter end date (YYYYMMDD): "))
      frequency = int(input("Enter recurrence frequency (e.g., 1 for daily, 7 for weekly): "))
      return {
        "task_name": task_name,
        "task_type": task_type,
        "start_time": start_time,
        "duration": duration,
        "start_date": start_date,
        "end_date": end_date,
        "frequency": frequency
      }
    elif task_type == "TransientTask":
      date = int(input("Enter date (YYYYMMDD): "))
      return {
        "task_name": task_name,
        "task_type": task_type,
        "start_time": start_time,
        "duration": duration,
        "date": date
      }
    elif task_type == "AntiTask":
      date = int(input("Enter date (YYYYMMDD): "))
      target_name = input("Enter the name of the recurring task to cancel: ")
      target_recurring_task = self.schedule.find_task(target_name)
      if not isinstance(target_recurring_task, RecurringTask):
        print("Error: Target task is not a recurring task or does not exist.")
        return None
      return {
        "task_name": task_name,
        "task_type": task_type,
        "start_time": start_time,
        "duration": duration,
        "date": date,
        "target_recurring_task": target_recurring_task
      }
    return None


  def prompt_for_file_name(self, action: str) -> str:
    return input(f"Enter the filename to {action} the schedule: ")

  def write_schedule_to_file(self, filename: str) -> None:
    if self.schedule.write_schedule_to_file(filename):
      self.task_view.display_success_message(f"Schedule saved to {filename}.")
    else:
      self.task_view.display_error_message("Failed to save schedule.")

  def read_schedule_from_file(self, filename: str) -> None:
    if self.schedule.read_schedule_from_file(filename):
      self.task_view.display_success_message(f"Schedule loaded from {filename}.")
    else:
      self.task_view.display_error_message("Failed to load schedule.")

  def view_schedule_by_period(self, start_date: int, period: str) -> None:
    tasks = self.schedule.view_tasks_by_period(start_date, period)
    print(f"\nSchedule for {period}:")
    if not tasks:
      print("No tasks found.")
      return

    for entry in tasks:
      if isinstance(entry, dict):  # For RecurringTask with occurrences
        task = entry["task"]
        occurrences = entry["occurrences"]
        print(f"{task.task_name} (RecurringTask)")
        print(f"  Start Time: {task.start_time} hrs")
        print(f"  Duration: {task.duration} hrs")
        print(f"  Occurrences in Range: {', '.join(occurrences)}")
      else:  # For TransientTask or similar
        TaskView.display_task_details(entry)

  # print out menu for UI
  def main_menu(self):
    while True:
      print("\n===== Personal Scheduling System (PSS) =====")
      print("1. Add Task")
      print("2. Edit Task")
      print("3. Find Task")
      print("4. View All Tasks")
      print("5. Save Schedule to File")
      print("6. Load Schedule from File")
      print("7. View Schedule by Period")
      print("8. Delete Task")
      print("9. Exit")
      choice = input("Choose an option: ")
      # option logic for menu
      if choice == "1":
        self.add_task()
      elif choice == "2":
        self.edit_task()
      elif choice == "3":
        self.find_task()
      elif choice == "4":
        self.view_tasks()
      elif choice == "5":
        filename = self.prompt_for_file_name("save")
        self.write_schedule_to_file(filename)
      elif choice == "6":
        filename = self.prompt_for_file_name("load")
        self.read_schedule_from_file(filename)
      elif choice == "7":
        start_date = int(input("Enter start date (YYYYMMDD): "))
        period = input("Enter period (day/week/month): ").lower()
        self.view_schedule_by_period(start_date, period)
      elif choice == "8":
        name = input("Enter the name of the task to delete: ")
        self.delete_task(name)
      elif choice == "9":
        print("Exiting the program. Goodbye!")
        break
      else:
        print("Invalid option. Please try again.")

## Main Program

In [8]:
# Initialize Components
schedule = Schedule()
task_view = TaskView()
schedule_view = ScheduleView()
controller = TaskController(schedule, task_view, schedule_view)

### Use Case 1: Create Task

In [9]:
# Run Program
controller.main_menu()


===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 1

-- Add Task --
1. Recurring Task
2. Transient Task
3. Anti-Task
Choose task type: 1
Enter task name: Meeting
Enter start time (e.g., 9.5 for 9:30 AM): 8
Enter duration (in hours): 1
Enter start date (YYYYMMDD): 20240101
Enter end date (YYYYMMDD): 20240202
Enter recurrence frequency (e.g., 1 for daily, 7 for weekly): 7
Success: Task created successfully.

===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 1

-- Add Task --
1. Recurring Task
2. Transient Task
3. Anti-Task
Choose task type: 3
Enter task name: Cancel meeting
Enter start time (e.g., 9.5 for 9:30 AM): 8
Enter duration (in hours): 1
Enter

### Use Case 2: Edit Task

In [10]:
# Run Program
controller.main_menu()


===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 2

-- Edit Task --
Enter the name of the task to edit: Shopping
Editing task: Shopping (TransientTask): 9.0 - 11.0 hrs
Enter task name: Night Shopping
Enter start time (e.g., 9.5 for 9:30 AM): 19
Enter duration (in hours): 3
Enter date (YYYYMMDD): 20240114
Success: Task edited successfully.

===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 4

-- All Tasks --

Schedule for all:
Meeting (RecurringTask)
  Start Date: January 01, 2024
  End Date: February 02, 2024
  Occurrences: January 01, 2024, January 15, 2024, January 22, 2024, January 29, 2024
  Start Time: 8.0 o'clock
  End Time: 9.0 o'clock
  Dur

### Use Case 3: Find Task

In [11]:
# Run Program
controller.main_menu()


===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 3

-- Find Task --
Enter the name of the task to find: Meeting
Task: Meeting (RecurringTask)
Start Date: January 01, 2024
End Date: February 02, 2024
Occurrences: January 01, 2024, January 15, 2024, January 22, 2024, January 29, 2024
Start Time: 8.0 o'clock
End Time: 9.0 o'clock
Duration: 1.0 hrs

===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 9
Exiting the program. Goodbye!


### Use Case 4: View Task

In [12]:
# Run Program
controller.main_menu()


===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 4

-- All Tasks --

Schedule for all:
Meeting (RecurringTask)
  Start Date: January 01, 2024
  End Date: February 02, 2024
  Occurrences: January 01, 2024, January 15, 2024, January 22, 2024, January 29, 2024
  Start Time: 8.0 o'clock
  End Time: 9.0 o'clock
  Duration: 1.0 hrs
Night Shopping (TransientTask) - January 14, 2024
  Start Time: 19.0 o'clock
  End Time: 22.0 o'clock
  Duration: 3.0 hrs

===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 9
Exiting the program. Goodbye!


### Use Case 5: Save Schedule to File

In [13]:
# Run Program
controller.main_menu()


===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 5
Enter the filename to save the schedule: schedule.json
Success: Schedule saved to schedule.json.

===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 9
Exiting the program. Goodbye!


### Use Case 6: Load Schedule from File

In [14]:
# Run Program
controller.main_menu()


===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 6
Enter the filename to load the schedule: schedule.json
Success: Schedule loaded from schedule.json.

===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 4

-- All Tasks --

Schedule for all:
Meeting (RecurringTask)
  Start Date: January 01, 2024
  End Date: February 02, 2024
  Occurrences: January 01, 2024, January 15, 2024, January 22, 2024, January 29, 2024
  Start Time: 8.0 o'clock
  End Time: 9.0 o'clock
  Duration: 1.0 hrs
Night Shopping (TransientTask) - January 14, 2024
  Start Time: 19.0 o'clock
  End Time: 22.0 o'clock
  Duration: 3.0 hrs

===== Personal Scheduling System (PSS) =====
1. Add 

### Use Case 7: View Schedule by Period

In [15]:
# Run Program
controller.main_menu()


===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 7
Enter start date (YYYYMMDD): 20240101
Enter period (day/week/month): month

Schedule for month:
Meeting (RecurringTask)
  Start Time: 8.0 hrs
  Duration: 1.0 hrs
  Occurrences in Range: January 01, 2024, January 15, 2024, January 22, 2024, January 29, 2024
Task: Night Shopping (TransientTask)
Date: January 14, 2024
Start Time: 19.0 o'clock
End Time: 22.0 o'clock
Duration: 3.0 hrs

===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 9
Exiting the program. Goodbye!


### Use Case 8: Delete Task

In [16]:
# Run Program
controller.main_menu()


===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 8
Enter the name of the task to delete: Night Shopping
Success: Task 'Night Shopping' deleted successfully.

===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedule by Period
8. Delete Task
9. Exit
Choose an option: 4

-- All Tasks --

Schedule for all:
Meeting (RecurringTask)
  Start Date: January 01, 2024
  End Date: February 02, 2024
  Occurrences: January 01, 2024, January 15, 2024, January 22, 2024, January 29, 2024
  Start Time: 8.0 o'clock
  End Time: 9.0 o'clock
  Duration: 1.0 hrs

===== Personal Scheduling System (PSS) =====
1. Add Task
2. Edit Task
3. Find Task
4. View All Tasks
5. Save Schedule to File
6. Load Schedule from File
7. View Schedul